In [ ]:
import os, sys
import tensorflow.compat.v1 as tf
sys.path.append('/home/ubuntu/IDP/waymo_open_dataset/')

import dataset_pb2 as open_dataset
import frame_utils
from pyquaternion import Quaternion
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw

In [ ]:
segment_path = '/home/ubuntu/IDP/waymo_open_dataset/dataset/val/segment-10203656353524179475_7625_000_7645_000_with_camera_labels.tfrecord'
if not os.path.exists(segment_path):
    raise ValueError('tfrecord for the segment doensn\'t exist')

In [ ]:
axes_limits = [
    [-80, 80], # X axis range
    [-80, 80], # Y axis range
    [-3, 10]   # Z axis range
]
axes_str = ['X', 'Y', 'Z']

colors = {
    0: 'b',
    1: 'r',
    2: 'g',
    3: 'c',
    4: 'm'
}

In [ ]:
# box = [label, length, width, height, x, y, z, heading]
def get_corners_from_labels_array(label, wlh_factor: float = 1.0) -> np.ndarray:
    ''' takes 1x8 array contains label information
    Args:
        np.array 1x8 contains label information [x, y, z, l, w, h, heading, labels]
    Returns:
    '''
    
    
    length = label[1] * wlh_factor
    width = label[2] * wlh_factor
    height = label[3] * wlh_factor
    
    
    
    # 3D bounding box corners. (Convention: x points forward, y to the left, z up.)
    x_corners = length / 2 * np.array([1, 1, 1, 1, -1, -1, -1, -1])
    y_corners = width / 2 * np.array([1, -1, -1, 1, 1, -1, -1, 1])
    z_corners = height / 2 * np.array([1, 1, -1, -1, 1, 1, -1, -1])
    corners = np.vstack((x_corners, y_corners, z_corners))
    
    orientation = Quaternion(axis=(0.0, 0.0, 1.0), radians=label[7])
    
    # Rotate
    corners = np.dot(orientation.rotation_matrix, corners)
    
    # Translate
    x, y, z = label[4], label[5], label[6]
    corners[0, :] = corners[0, :] + x
    corners[1, :] = corners[1, :] + y
    corners[2, :] = corners[2, :] + z
    
    return corners

In [ ]:
def draw_box(pyplot_axis, vertices, axes=[0, 1, 2], color='black'):
    """
    Draws a bounding 3D box in a pyplot axis.
    
    Parameters
    ----------
    pyplot_axis : Pyplot axis to draw in.
    vertices    : Array 8 box vertices containing x, y, z coordinates.
    axes        : Axes to use. Defaults to `[0, 1, 2]`, e.g. x, y and z axes.
    color       : Drawing color. Defaults to `black`.
    """
    vertices = vertices[axes, :]
    connections = [
        [0, 1], [1, 2], [2, 3], [3, 0],  # Lower plane parallel to Z=0 plane
        [4, 5], [5, 6], [6, 7], [7, 4],  # Upper plane parallel to Z=0 plane
        [0, 4], [1, 5], [2, 6], [3, 7]  # Connections between upper and lower planes
    ]
    for connection in connections:
        pyplot_axis.plot(*vertices[:, connection], c=color, lw=0.5)

In [ ]:
def draw_point_cloud(points_all, labels_arr, ax, title, axes=[0, 1], xlim3d=None, ylim3d=None, zlim3d=None):
        """
        Convenient method for drawing various point cloud projections as a part of frame statistics.
        """
        # plot point cloud
        ax.scatter(*np.transpose(points_all[:, axes]),s=0.02, cmap='gray')
        ax.set_title(title)
        ax.set_xlabel('{} axis'.format(axes_str[axes[0]]))
        ax.set_ylabel('{} axis'.format(axes_str[axes[1]]))
        ax.set_xlim(*axes_limits[axes[0]])
        ax.set_ylim(*axes_limits[axes[1]])
        # User specified limits
        if xlim3d!=None:
            ax.set_xlim3d(xlim3d)
        if ylim3d!=None:
            ax.set_ylim3d(ylim3d)
        if zlim3d!=None:
            ax.set_zlim3d(zlim3d)
            
        for i in range(labels_arr.shape[0]):
            label_type = labels_arr[i][0] # get label
#             box_corners = np.transpose(t[1])
#             box_corners = t[1]
#             print(box_corners)
            box_corners = get_corners_from_labels_array(labels_arr[i])
            draw_box(ax, box_corners, axes=axes, color=colors[label_type])
#             break

In [ ]:
# Read TFRecord
recorded_segment = tf.data.TFRecordDataset(segment_path, compression_type='')
# Loop over every frame
frames = 0
for data in recorded_segment:
    # Read the first frame only
    frame = open_dataset.Frame()
    frame.ParseFromString(bytearray(data.numpy()))

    # extract the camera images, camera projection points and range images
    (range_images, 
    camera_projections, 
    range_image_top_pose) = frame_utils.parse_range_image_and_camera_projection(frame)

    # First return of Lidar data
    points, cp_points = frame_utils.convert_range_image_to_point_cloud(
        frame,
        range_images,
        camera_projections,
        range_image_top_pose)

    # concatenate all LIDAR points from the 5 radars.
    points_all = np.concatenate(points, axis=0)

    bboxes = []
    for laser_label in frame.laser_labels:
        label = laser_label.type
        length = laser_label.box.length
        width = laser_label.box.width
        height = laser_label.box.height
        x, y, z = laser_label.box.center_x, laser_label.box.center_y, laser_label.box.center_z
        heading = laser_label.box.heading
        box = [label, length, width, height, x, y, z, heading]
        bboxes.append(box)
        
    labels_arr = np.array(bboxes, dtype=np.float32)

    # Plotting 
    f, ax3 = plt.subplots(1, 1, figsize=(25, 25))
    draw_point_cloud(points_all, 
                     labels_arr, 
                     ax3, 
                     'Waymo segment, XY projection (Z = 0), the car is moving in direction left to right', 
                     axes=[0, 1]) # X and Y axes
    plt.savefig('_frame_{}_projection.png'.format(frames))
    frames += 1
    
    break #TODO: remove later

In [ ]:
import glob
img_list = sorted(glob.glob( '/home/ubuntu/IDP/notebooks' + '/*'+'.png', recursive=True))

In [ ]:
img_list

In [ ]:
images = []
for img in img_list:
    im = Image.open(img)
    images.append(im)

In [ ]:
im.save('out.gif', save_all=True, append_images=images)